In [21]:
from datasets import load_dataset
import zipfile
import torch
import io
from PIL import Image
import numpy as np
import pandas as pd

meta_data = pd.read_csv("diffusiondb.csv")
for index, filepath in enumerate(meta_data['filepath']):
    meta_data.loc[index, 'filepath'] = filepath.split("/")[-1]

# print(meta_data['prompt'][0])
# print(meta_data['filepath'][0])
meta_data.head()

,filepath,prompt
0,2217ccbd-a1c6-47ac-9a2d-79649727c834.png,"a portrait of a female robot made from code, v..."
1,48eb7e17-a3cf-4eb8-96a9-d8e3e23fa1af.png,dream swimming pool with nobody
2,2919b048-6f68-4ac7-a6d5-060d827abb77.png,a beautiful paint of cultists dancing surround...
3,3c835fdc-9047-4298-ac8a-7461f5490132.png,"frontal portrait of ragged, worried twin women..."
4,454d7550-a6cf-4896-befb-e2449b281265.png,a stunning portrait of an asian samurai with l...


In [69]:
from datasets import load_dataset, dataset_dict
import zipfile
import torch
import io
from PIL import Image
import numpy as np

zip_data_path ="diffusiondb-filtered-0-40000.zip"
X = torch.tensor([[[[]]]], dtype=torch.int8)
X = torch.zeros(size=(1, 512,512,3))
Y = []
with zipfile.ZipFile(zip_data_path, "r") as zip_data:
    content_list = zip_data.namelist()
    for index, name_file in enumerate(content_list):
        
        img_bytes = zip_data.open(name_file)          
        img_data = Image.open(img_bytes)              
        image_as_array = np.array(img_data, np.uint8)
        image_as_array = torch.from_numpy(image_as_array)
        image_as_array.unsqueeze_(dim=0)
        # print(image_as_array)
        # print(image_as_array.shape)
        print(meta_data[meta_data.eq(name_file.split("/")[-1]).any(1)]["prompt"].item() is str)
        Y.append(meta_data[meta_data.eq(name_file.split("/")[-1]).any(1)]["prompt"].item())
        X = torch.cat([X, image_as_array], dim=0)
        # X = np.append(X, image_as_array)
        if index == 100:
            break
# X: 이미지 데이터 배열, Y: 프롬프트 데이터 배열
print(X.shape, len(X))
X = X[1:]
print(X[0])
# X = [Image.open(archive.read(file.filename)) for file in file_list]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
torch.Size([102, 512, 512, 3]) 102
tensor([[[ 57.,  44.,  20.],
         [ 68.,  61.,  25.],
         [ 62.,  51.,  25.],
         ...,
         [164., 116.,  16.],
         [180., 125.,  28.],
         [157., 101.,  33.]],

        [[ 43.,  32.,  10.],
         [ 39.,  25.,   6.],
         [ 36.,  24.,   7.],
         ...,
         [181., 143.,  19.],
         [199., 158.,  22.],
         [

In [71]:
print(type(Y[1]))
print(Y[1])
print(X.shape)

<class 'str'>
A Singaporean propaganda poster featuring Lee Kuan Yew designed by Saul Bass
torch.Size([101, 512, 512, 3])


In [72]:
from torch.utils.data import Dataset

class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        encoding = self.processor(images=item["image"], text=item["text"], padding="max_length", return_tensors="pt")

        # remove batch dimension
        encoding = {k:v.squeeze() for k,v in encoding.items()}

        return encoding

In [73]:
from PIL import Image
import requests
from transformers import AutoProcessor, GitVisionModel

processor = AutoProcessor.from_pretrained("microsoft/git-base")


train_data = []
for i in range(len(X)):
    train_data.append({
        "image": X[i],
        "text":Y[i]
    })

train_dataset = ImageCaptioningDataset(train_data, processor)
item = train_dataset[0]

In [76]:
item = train_dataset[0]
for k,v in item.items():
  print(k,v.shape)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)
  
processor.decode(batch["input_ids"][0])

input_ids torch.Size([512])
attention_mask torch.Size([512])
pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([2, 512])
attention_mask torch.Size([2, 512])
pixel_values torch.Size([2, 3, 224, 224])


'[CLS] nicolas cage with a bird for hair [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")

outputs = model(input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                pixel_values=batch["pixel_values"],
                labels=batch["input_ids"])
outputs.loss